In [1]:
# AutoEncoder
# https://medium.com/analytics-vidhya/image-anomaly-detection-using-autoencoders-ae937c7fd2d1

# ogólne
import tensorflow as tf
import numpy as np

# ładowanie danych
from keras.preprocessing.image import ImageDataGenerator

# budowanie modelu
from keras.layers import LeakyReLU, Conv2D, BatchNormalization, Conv2DTranspose
from keras.optimizers import Adam
from keras import Model, Input

# proces uczenia
from keras.callbacks import ModelCheckpoint, EarlyStopping
from datetime import datetime

# wizualizacja
import matplotlib.pyplot as plt
from keras.utils import load_img
import pickle

# klasyfikacja
from keras.models import load_model

# https://stackoverflow.com/questions/58352326/running-the-tensorflow-2-0-code-gives-valueerror-tf-function-decorated-functio
tf.config.run_functions_eagerly(True)  
# tf.data.experimental.enable_debug_mode()

In [41]:
# Przygotowanie danych
# https://stackoverflow.com/questions/42443936/keras-split-train-test-set-when-using-imagedatagenerator

path = r"C:\Users\User\Desktop\Matematyka stosowana - II stopień\III semestr\Uczenie ze wzmocnieniem i deeplearning\deep_learning\Zadanie 1"
nbatch = 64

datagen = ImageDataGenerator(validation_split=0.15,
                             rescale=1./255,
                             rotation_range=10.,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.2,
                            #  data_format='channels_last',
                             horizontal_flip=True)

train_data = datagen.flow_from_directory(directory=path+r"\train",
                                         target_size=(224,224),
                                         shuffle=True,
                                         color_mode="grayscale",
                                         batch_size=nbatch,
                                         class_mode="input",
                                        #  classes=None,
                                         subset='training') # set as training data


valid_data = datagen.flow_from_directory(directory=path+r"\train",
                                         target_size=(224,224),
                                         shuffle=True,
                                         color_mode="grayscale",
                                         batch_size=nbatch,
                                         class_mode="input",
                                        #  classes=None,
                                         subset='validation') # set as validation data

Found 890 images belonging to 1 classes.
Found 157 images belonging to 1 classes.


In [32]:
inputs = Input(shape=(224, 224, 3), name='input_layer')

# Conv Block 1 -> BatchNorm->leaky Relu
encoded = Conv2D(filters=32, kernel_size=(3,3), strides=2, padding='same', name='conv_1')(inputs)
encoded = BatchNormalization(name='batchnorm_1')(encoded)
encoded = LeakyReLU(name='leaky_relu_1')(encoded)
# Conv Block 2 -> BatchNorm->leaky Relu
# encoded = Conv2D(filters=64, kernel_size=(3,3), strides=2, padding='same', name='conv_2')(encoded)
# encoded = BatchNormalization(name='batchnorm_2')(encoded)
# encoded = LeakyReLU(name='leaky_relu_2')(encoded)
# # Conv Block 3 -> BatchNorm->leaky Relu
# encoded = Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), padding='same', name='conv_3')(encoded)
# encoded = BatchNormalization(name='batchnorm_3')(encoded)
# encoded = LeakyReLU(name='leaky_relu_3')(encoded)

encoder = Model(inputs, encoded)

def SSIMLoss(y_true, y_pred):
  return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))
encoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0005), loss=[SSIMLoss])

encoder.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 conv_1 (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 batchnorm_1 (BatchNormaliza  (None, 112, 112, 32)     128       
 tion)                                                           
                                                                 
 leaky_relu_1 (LeakyReLU)    (None, 112, 112, 32)      0         
                                                                 
Total params: 1,024
Trainable params: 960
Non-trainable params: 64
_________________________________________________________________


In [33]:
hist=encoder.fit(x=train_data,
                 epochs=10,
                 batch_size=nbatch,
                 shuffle=True,
                 validation_data=valid_data)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\User\AppData\Local\Temp\ipykernel_83696\720978468.py", line 19, in SSIMLoss  *
        return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))

    ValueError: Shapes (224, 224, 3) and (112, 112, 32) are incompatible


In [42]:
# Create the Encoder and Decoder
#pass the gray scale input image of size(28,28,1)
inputs = Input(shape=(224, 224, 1), name='input_layer')
# Conv Block 1 -> BatchNorm->leaky Relu
encoded = Conv2D(32, kernel_size=(3,3), strides= 1, padding='same', name='conv_1')(inputs)
encoded = BatchNormalization(name='batchnorm_1')(encoded)
encoded = LeakyReLU(name='leaky_relu_1')(encoded)
# Conv Block 2 -> BatchNorm->leaky Relu
encoded = Conv2D(64, kernel_size=(3,3), strides= 2, padding='same', name='conv_2')(encoded)
encoded = BatchNormalization(name='batchnorm_2')(encoded)
encoded = LeakyReLU(name='leaky_relu_2')(encoded)
# Conv Block 3 -> BatchNorm->leaky Relu
encoded = Conv2D(64, kernel_size=(3,3), strides=2, padding='same', name='conv_3')(encoded)
encoded = BatchNormalization(name='batchnorm_3')(encoded)
encoded = LeakyReLU(name='leaky_relu_3')(encoded)
#Decoder
# DeConv Block 1-> BatchNorm->leaky Relu
decoded = Conv2DTranspose(64, 3, strides= 1, padding='same',name='conv_transpose_1')(encoded)
decoded = BatchNormalization(name='batchnorm_4')(decoded)
decoded = LeakyReLU(name='leaky_relu_4')(decoded)
# DeConv Block 2-> BatchNorm->leaky Relu
decoded = Conv2DTranspose(64, 3, strides= 2, padding='same', name='conv_transpose_2')(decoded)
decoded = BatchNormalization(name='batchnorm_5')(decoded)
decoded = LeakyReLU(name='leaky_relu_5')(decoded)
# DeConv Block 3-> BatchNorm->leaky Relu
decoded = Conv2DTranspose(32, 3, 2, padding='same', name='conv_transpose_3')(decoded)
decoded = BatchNormalization(name='batchnorm_6')(decoded)
decoded = LeakyReLU(name='leaky_relu_6')(decoded)
# output
outputs = Conv2DTranspose(1, 3, 1,padding='same', activation='sigmoid', name='conv_transpose_4')(decoded)

def SSIMLoss(y_true, y_pred):
  return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0))

autoencoder = tf.keras.Model(inputs, outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005)
autoencoder.compile(optimizer=optimizer, loss=SSIMLoss)
autoencoder.summary()

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 1)]     0         
                                                                 
 conv_1 (Conv2D)             (None, 224, 224, 32)      320       
                                                                 
 batchnorm_1 (BatchNormaliza  (None, 224, 224, 32)     128       
 tion)                                                           
                                                                 
 leaky_relu_1 (LeakyReLU)    (None, 224, 224, 32)      0         
                                                                 
 conv_2 (Conv2D)             (None, 112, 112, 64)      18496     
                                                                 
 batchnorm_2 (BatchNormaliza  (None, 112, 112, 64)     256       
 tion)                                                    

In [43]:
hist=autoencoder.fit(x=train_data,
                     epochs=10,
                     batch_size=nbatch,
                     shuffle=True,
                     validation_data=valid_data)

Epoch 1/10
11/14 [======================>.......] - ETA: 48s - loss: 0.6344 

KeyboardInterrupt: 